# INTRODUCTION

## 1. Define problem

As you know, we are living in an era of digital data abundance and technical involving extracting knowledge or useful insights from those data is necessary for adoption intelligent solutions in various application areas. Thus, the data science industry is playing an increasingly important role, and its salaries also are becoming more and more attractive. 
In this analysis, I will: 
- Explore the factors directly impacting data science industry salaries over the past 4 years.
- Train a model with these factors to predict the salary in near future.

## 2. Link for dataset

 https://www.kaggle.com/datasets/iamsouravbanerjee/data-science-salaries-2023

## 3. Features of dataset

1. job_title: Data related professions
2. employment_type: Type of employment
    - Full-Time
    - Part-Time
    - Contract
    - Freelance
3. experience_level: Experience Level
    - Entry
    - Medium
    - Senior
    - Executive
4. company_location: Country of the employer's head office or branch office
5. company_size: Average number of employees during the year
    - Small: less than 50 employees
    - Medium: from 50 to 250 employees
    - Large: more than 250 employees
6. year: The year in which the salary was paid
    - 2020
    - 2021
    - 2022
    - 2023
7. salary_in_usd: Salaries in US dollars (currency rate divided by the average US dollar rate for the corresponding year via fxdata.foorilla.com)

## 4. Import neccessary libraries for processing

In [1]:
# for basics purposes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statistics

# for machine learning
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# for evaluation model
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, \
                            median_absolute_error, max_error,confusion_matrix, ConfusionMatrixDisplay
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

# for deep learning
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# PART 1

In [2]:
# Import data and exclude redundant columns
sal = pd.read_csv('Latest_Data_Science_Salaries.csv')
sal.columns = sal.columns.str.lower()
sal.columns = sal.columns.str.replace(" ","_")
sal.drop(['expertise_level', 'salary','salary_currency', 'employee_residence'], axis = 1, inplace = True)
sal.head()

In [3]:
# Let's check the dtype of each column
sal.dtypes

In [4]:
# We see that some title written in full name, but some in abbreviated name
a = sal['job_title'].unique()
np.sort(a)

In [5]:
sal.job_title.nunique()

In [6]:
# So we need to change some full-name titles to abbreviated name

bi_da_an = ['BI Analyst', 'Business Intelligence Analyst', 'Business Intelligence Data Analyst'] #change to 'BI Data Analyst'
bi_da_en = ['Business Intelligence Engineer'] #change to 'BI Data Engineer'
da_en = ['Data Engineer 2'] # change to 'Data Engineer'
da_mo = ['Data Modeller'] # change to 'Data Modeler'
fi_da_an = ['Finance Data Analyst'] # change to 'Financial Data Analyst'
he_of_da = ['Head of Data Science'] # change to 'Head of Data'
ml_en = ['Machine Learning Engineer'] # change to 'ML Engineer'

change = {'BI Data Analyst':bi_da_an,
          'BI Data Engineer':bi_da_en,
          'Data Engineer':da_en,
          'Data Modeler':da_mo,
          'Financial Data Analyst':fi_da_an,
          'Head of Data':he_of_da,
          'ML Engineer':ml_en}

In [7]:
# We create function to change title easily
def change_title(changefrom, changeto):
    indexes = []
    for word in changefrom:
        ind = sal[sal['job_title'].str.contains(word)].index
        indexes.extend(ind)
    
    sal.loc[indexes,'job_title'] = changeto
    return sal

for name, values in change.items():
    change_title(changefrom = values, changeto = name)

In [8]:
# Now we just only have 98 job titles
sal.job_title.nunique()

In [9]:
cop = sal.copy()

In [10]:
# Let's see the distributon of salary 

min_sal = min(cop['salary_in_usd'])
max_sal = max(cop['salary_in_usd'])
print('Minimum salary: ', min_sal)
print('Maximum salary: ', max_sal)

a = [round(q) for q in statistics.quantiles(cop['salary_in_usd'], n=4)]
quan = pd.DataFrame({'Percent': ['25%', '50%', '75%'], 
                     'Salary': a})
print('\nInterquantile of the salary')
quan

In [11]:
# Plot the salary distributon 

sns.color_palette("muted")

#Create histogram
fig, ax1 = plt.subplots(figsize = (10, 5))
ax1.hist(cop['salary_in_usd'], bins = 30, color = 'forestgreen')

#Create the density
ax2 = ax1.twinx()
ax2.set_ylabel('Salary Density')
sns.kdeplot(cop['salary_in_usd'], color = 'darkgreen')

# Line 25%
ax1.axvline(x = 90000, linestyle = '-', color = 'black', ls ='--')
ax1.annotate('25% vacancies \n have a salary \n less than $90,000',
             xy = (45000, 260),
             color = 'black', ha="center", va="center")

# Line 75%
ax1.axvline(x = 185000, linestyle = '-', color = 'black', ls ='--')
ax1.annotate('25% vacancies \n have a salary \n more than $185,000',
             xy = (235000, 260),
             color = 'black', ha="center", va="center")

ax1.set_ylabel('Number of vacancies')
ax1.set_xlabel('Salary in data science industry')
ax1.grid(False)
ax2.grid(False)
plt.title('Salary distribution', fontsize = 13, fontweight = 'bold')
plt.xlim(0,480000)

In [12]:
# Let's take the titles of the most 15 job in-demand
top15 = cop['job_title'].value_counts(sort=True)[:15]

#create df of top 15
df_15 = cop[cop['job_title'].isin(top15.index)]
sal_job15 = df_15.groupby(by = 'job_title').agg({'salary_in_usd': 'mean','job_title': 'count'} ) 
sal_job15['salary_in_usd'] = [round(i) for i in sal_job15['salary_in_usd']]
sal_job15.columns = ['Salary in year', 'Number of vacancies']
sal_job15.sort_values(by = 'Number of vacancies', ascending = False, inplace = True)

# sal_job15.drop('year', axis = 1, inplace = True)

sal_job15

In [13]:
# Create subplots

fig, (ax1,ax2) = plt.subplots(figsize = (15,7),
                              sharex = True, 
                              nrows = 2, 
                              ncols = 1)

## The first plot
ax1.bar(height = sal_job15['Number of vacancies'] , 
        x = sal_job15.index, 
        color = 'forestgreen')

ax1.set_ylabel('Number of jobs')
ax1.set_title('Number of jobs and wages by “Position”', fontsize = 13, fontweight = 'bold')

## The second plot
sns.boxplot(y = df_15['salary_in_usd'],
            x = df_15['job_title'], 
            order = sal_job15.index,  
            ax = ax2,
            palette = sns.color_palette("Paired"))

ax2.set_ylabel('Salary in a year ($)')
ax2.set_xlabel(None)

ax1.grid(False)
ax2.grid(False)
plt.xticks(rotation = 75)
plt.show()

In [14]:
# Change type of dataframe
cat_cols = ['job_title', 'employment_type', 'experience_level', 'company_location', 'company_size']
for col in cat_cols:
    cop[col] = cop[col].astype('category')

cop['year'] = cop['year'].astype('string') # we change type of column year to string so that xticks won't be decimal number  
cop.dtypes

In [15]:
# Number of jobs and salary of top 10 by Company Location

## Create data frame
asd = cop.groupby('company_location').agg({'salary_in_usd':'mean', 'company_location':'count'})
asd.columns = ['salary', 'location']
asd.sort_values(by= 'location', ascending = False, inplace = True)
asd = asd.iloc[:10]

## Plot
fig, ax1 = plt.subplots(figsize = (15, 7))
plt.title('Number of jobs and wages by “Company location”', fontsize = 12, fontweight = 'bold')


ax1.bar(x = asd.index, height = asd['location'], color='forestgreen')
ax1.set_ylabel('Number of jobs')
ax1.tick_params(axis='y', labelcolor='black')


ax2 = ax1.twinx()
ax2.plot(asd.index, asd['salary'], color = 'black', marker = 'o')
ax2.set_ylabel('Salary in a year ($)', color='black')
ax2.tick_params(axis='y', labelcolor='black')
ax2.legend('$')

ax1.grid(False, axis = 'both')
ax2.grid(False, axis = 'both')
plt.show()

In [16]:
# Plot for 4 variables by Number of vacancies

## Create data
exp = cop['experience_level'].value_counts()
emp = cop['employment_type'].value_counts()
comsize = cop['company_size'].value_counts()
yer = cop['year'].value_counts()


## Plot 
fig, ((ax1,ax2),(ax3, ax4)) = plt.subplots(nrows = 2, ncols = 2, figsize = (15,7))
fig.suptitle('Number of vacancies by four factors', 
             fontsize = 16, fontweight = 'bold')


ax1.bar(height = emp, x = emp.index, color = 'forestgreen')
ax1.set_ylabel('Number of vacancies')
ax1.set_title('by “Type of employment”', fontsize = 14, fontweight = 'bold')
ax1.grid(False)
for i in range(len(emp.index)):
    ax1.text(i-0.1,emp[i]+25,emp[i], fontsize = 9)


ax2.bar(height = exp, x = exp.index, color = 'forestgreen')
ax2.set_ylabel('Number of vacancies')
ax2.set_title('by "Experience Level"', fontsize = 14, fontweight = 'bold')
ax2.grid(False)
for i in range(len(exp.index)):
    ax2.text(i-0.1,exp[i]+15,exp[i], fontsize = 9)
    
    
ax3.bar(height = comsize, x = comsize.index, color = 'forestgreen')
ax3.set_ylabel('Number of vacancies')
ax3.set_title('by "Company Size"', fontsize = 14, fontweight = 'bold')
ax3.grid(False)
for i in range(len(comsize.index)):
    ax3.text(i-0.07,comsize[i]+15,comsize[i], fontsize = 9)

    
ax4.bar(height = yer, x = yer.index, color = 'forestgreen')
ax4.set_ylabel('Number of vacancies')
ax4.set_title('by "Year"', fontsize = 14, fontweight = 'bold')
ax4.grid(False)
for i in range(len(yer.index)):
    ax4.text(i-0.1,yer[i]+15,yer[i], fontsize = 9)

plt.show()

In [17]:
# Plot for 4 variables by Salary
fig, ((ax1,ax2),(ax3, ax4)) = plt.subplots(nrows = 2, ncols = 2, figsize = (15,7))
fig.suptitle('Salary by four factors', fontsize = 16, fontweight = 'bold')

sns.boxplot(data = cop, y = 'salary_in_usd', x = 'employment_type', palette = sns.color_palette("Paired"), 
            order = ['Full-Time', 'Contract', 'Part-Time',  'Freelance'], ax = ax1)
ax1.set_ylabel('Salary in a year ($)')
ax1.set_xlabel(None)
ax1.set_yticks(np.arange(0, 500000, step=50000))
ax1.set_title('by "Type of Employment"', fontsize = 14, fontweight = 'bold')
ax1.grid(False)

sns.boxplot(data = cop, y = 'salary_in_usd', x = 'experience_level',palette = sns.color_palette("Paired"),
            order = [ 'Entry', 'Mid','Senior', 'Executive'], ax = ax2)
ax2.set_ylabel('Salary in a year ($)')
ax2.set_xlabel(None)
ax2.set_yticks(np.arange(0, 500000, step=50000)) 
ax2.set_title('by "Level of Experience"', fontsize = 14, fontweight = 'bold')
ax2.grid(False)


sns.boxplot(data = cop, y = 'salary_in_usd', x = 'company_size', 
            palette = sns.color_palette("Paired"), order = ['Small', 'Medium', 'Large'], ax = ax3)
ax3.set_xlabel(None)
ax3.set_ylabel('Salary in a year ($)')
ax3.set_title('by "Size of Company"', fontsize = 14, fontweight = 'bold')
ax3.grid(False)


sns.boxplot(data = cop, y = 'salary_in_usd', x = 'year',
            palette = sns.color_palette("Paired"), order = ['2020', '2021', '2022', '2023'], ax = ax4)
ax4.set_xlabel(None)
ax4.set_ylabel('Salary in a year ($)', fontsize = 12)
ax4.set_title('by "Year"', fontsize = 14, fontweight = 'bold')
ax4.grid(False)

plt.show()

# PART 2

In [18]:
# Create function for test Kruskal-Wallis
def kruskal(variable):
    unique = cop[variable].unique()
    num_unique = cop[variable].nunique()
    if num_unique == 3:
        var0 = cop['salary_in_usd'][cop[variable] == unique[0]].to_list()
        var1 = cop['salary_in_usd'][cop[variable] == unique[1]].to_list()
        var2 = cop['salary_in_usd'][cop[variable] == unique[2]].to_list()
        result = stats.kruskal(var0, var1, var2)

    elif num_unique == 4:
        var0 = cop['salary_in_usd'][cop[variable] == unique[0]].to_list()
        var1 = cop['salary_in_usd'][cop[variable] == unique[1]].to_list()
        var2 = cop['salary_in_usd'][cop[variable] == unique[2]].to_list()
        var3 = cop['salary_in_usd'][cop[variable] == unique[3]].to_list()
        result = stats.kruskal(var0, var1, var2,var3)
    
    # Print the result
    print('\nThe variable', str.upper(variable), 'has result:')
    print(result)

In [19]:
kruskal('year'), kruskal('company_size'), kruskal('employment_type'), kruskal('experience_level')

# PART 3

In [20]:
copp = cop.copy()
copp.head()

In [21]:
copp.dtypes

In [23]:
# Change all category columns to numeric
to_onehot = ['employment_type', 'experience_level', 'company_size', 'year','job_title', 'company_location']

onehot = OneHotEncoder(sparse_output = False)
trans = ColumnTransformer([('onehot', onehot, to_onehot)], 
                          remainder = 'passthrough')
copp_trans = trans.fit_transform(copp)
copp_df = pd.DataFrame(copp_trans)

In [24]:
copp_df

In [26]:
X = copp_df.drop(184, axis = 1)
y = copp_df[184]

np.random.seed(100)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)
len(X_train), len(X_test)

In [27]:
models = {'Ridge': Ridge(max_iter = 3000, alpha = 4.0, solver='sag'),
         'Lasso': Lasso(max_iter = 3000, alpha = 4.0),
         'ElasticNet': ElasticNet(),
         'RandomForestRegressor': RandomForestRegressor()}
for name, model in models.items():
    model_use = model
    model_use.fit(X_train, y_train)
    score = model_use.score(X_test, y_test)
    print(f'{name} has score: {score :.5f}')

In [28]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
model.score(X_test, y_test)
y_preds = model.predict(X_test)

In [29]:
print(f' Value of r2_score: {r2_score(y_test, y_preds) :.3f} \n \
Value of mean_absolute_error: {mean_absolute_error(y_test, y_preds) :.3f} \n \
Value of mean_squared_error : {mean_squared_error(y_test, y_preds) :.3f} \n \
Value of median_absolute_error: {median_absolute_error(y_test, y_preds) :.3f} \n \
Value of cross_val_score: {np.mean(cross_val_score(model, X_train, y_train, cv=5)) :.3f}')

In [30]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
scores = cross_val_score(model, X_train, y_train, cv=5)
scores.mean()

In [31]:
model.feature_importances_

### Enhance model

In [32]:
pagram_grid = {'n_estimators': [5,10,15],
               'min_samples_split':[2,3,4],
               'n_jobs':[1,2,3]}

base_estimator = RandomForestRegressor(verbose=2,max_features='log2', criterion = 'absolute_error')

sh = HalvingGridSearchCV(base_estimator, pagram_grid, cv = 5,
                         factor = 3, max_resources = 20).fit(X_train, y_train)
sh.score(X_test, y_test)

In [33]:
sh.best_estimator_

Because the score after using HalvingGridSearchCV is not good as default parameters, we use default model

## Classification method by separating salary to some intervals

In [34]:
# Use Silhouette method to decide number of clusters
fig, ax = plt.subplots(4, 2, figsize=(15,8))
for i in range(2,10):
 
    # Create KMeans instances for different number of clusters
    km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=100, random_state=42)
    q, mod = divmod(i, 2)
    
    # Create SilhouetteVisualizer instance
    visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(X) 

In [35]:
# Additionaly, use Elbow method to decide number of clusters
from yellowbrick.cluster import KElbowVisualizer

# Instantiate the clustering model and visualizer
model = KMeans(init='k-means++', n_init=10, max_iter=100, random_state=42)
visualizer = KElbowVisualizer(
    model, k=(2,10), metric='distortion', timings=False)

visualizer.fit(X)        
visualizer.show()   

Despite showing optimized point at 4, we realize that sum of variances also can reduce when the number of cluster increases. Combining with Silhouette method, I decide to choose n_cluster = 6, where the red line goes through all classes.

In [36]:
# This code is copied from Sklearn documentation

reduced_data = PCA(n_components=2).fit_transform(X)
kmeans = KMeans(init="k-means++", n_clusters=6, n_init=10)
kmeans.fit(reduced_data)

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = 0.02  # point in the mesh [x_min, x_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1)
plt.clf()
plt.imshow(
    Z,
    interpolation="nearest",
    extent=(xx.min(), xx.max(), yy.min(), yy.max()),
    cmap=plt.cm.Paired,
    aspect="auto",
    origin="lower",
)

plt.plot(reduced_data[:, 0], reduced_data[:, 1], "k.", markersize=2)
# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(
    centroids[:, 0],
    centroids[:, 1],
    marker="x",
    s=169,
    linewidths=3,
    color="w",
    zorder=10,
)
plt.title(
    "K-means clustering (PCA-reduced data)\n"
    "Centroids are marked with white cross"
)
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()

In [37]:
(np.max(copp.salary_in_usd)-np.min(copp.salary_in_usd))/6

In [38]:
interval = []
for i in range(7):
    x = 72500*i+15000
    interval.append(x)
interval

In [39]:
# for interval 6
def cat_sal(x):
    if (x >= 15000 and x < 87500):
        return  '15000-87500'
    elif (x >= 87500 and x < 160000):
        return  '87500-160000'
    elif (x >= 160000 and x < 232500):
        return  '160000-232500'
    elif (x >= 232500 and x < 305000):
        return  '232500-305000'
    elif (x >= 305000 and x < 377500):
        return  '305000-377500'
    else: return '377500-450000'

In [40]:
# Create new column "cat_sal" for group of salary
copp_df['cat_sal'] = copp_df[184].apply(cat_sal)
copp_df = copp_df.drop(184, axis = 1)

In [41]:
# Split data for ML
X = copp_df.drop('cat_sal', axis = 1)
y = copp_df['cat_sal']

np.random.seed(100)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)
len(X_train), len(X_test)

In [42]:
models = {'DecisionTreeClassifier': DecisionTreeClassifier(),
         'RandomForestClassifier': RandomForestClassifier(),
         'SVC': SVC(),
         'LinearDiscriminantAnalysis': LinearDiscriminantAnalysis(n_components=2)}

for name, model in models.items():
    model_use = model
    model_use.fit(X_train, y_train)
    score = model_use.score(X_test, y_test)
    print(f'{name} has score: {score :.5f}')

Although model RandomForestClassifier has higher point than the model created before, the score is still not really good. Let's try model in Deep Learning

## Deep Learning 

In [43]:
# Change string type to numeric type
le = LabelEncoder()
y_le = le.fit_transform(y)

# Convert value to torch
X_tensor = torch.tensor(X.values).float()
y_tensor = torch.tensor(y_le, dtype = torch.long)
y_tensor = y_tensor[:, None]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size= 0.2)

# Create train and test dataset
train_tensor = TensorDataset(X_train, y_train)
test_tensor = TensorDataset(X_test, y_test)

# Load data
train_loader = DataLoader(train_tensor, batch_size=128)
test_loader = DataLoader(test_tensor, batch_size = test_tensor.tensors[0].shape[0])

In [44]:
# Create model with a few hidden layers
model = nn.Sequential(nn.Linear(184,150),
                      nn.ReLU(),
                      nn.Linear(150,100),
                      nn.ReLU(),
                      nn.Linear(100,70),
                      nn.ReLU(),
                      nn.Linear(70,40),
                      nn.ReLU(),
                      nn.Linear(40,20),
                      nn.ReLU(),
                      nn.Linear(20,6),
                      nn.Softmax(), )

# Loss function
lossfunc = nn.CrossEntropyLoss()

# Optimize
optimizer = torch.optim.SGD(model.parameters(), lr = 0.2, weight_decay = 0.2 )

In [45]:
model

In [46]:
# Train model
num_epoches = 100
trainAcc = []
testAcc = []

for epoch in range(num_epoches):
    batchAcc = []
    batchLoss = []
    for X_use, y_use in train_loader:
        #forward step
        yHat = model(X_use) 
        y_use = y_use.squeeze_() # to advoid error of multi-target
        loss = lossfunc(yHat, y_use)
        
        # backward step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batchAcc.append(100 * torch.mean( (torch.argmax(yHat, axis = 1) == y_use).float() ).item() )
        
    trainAcc.append(np.mean(batchAcc))
    X_test, y_test = next(iter(test_loader))
    y_preds = model(X_test)
    testAcc.append(100 * torch.mean( (torch.argmax(y_preds, axis = 1) == y_test).float() ).item() )

In [47]:
print('Total Train Accuracy: ', np.mean(trainAcc))
print('Total Test Accuracy: ', np.mean(testAcc))
fig, ax = plt.subplots(nrows=1, ncols=1, figsize = (10,5))

ax.plot(trainAcc, label = 'train', linewidth = 2.0)
ax.plot(testAcc, label = 'test' , linewidth = 1.0)
ax.set_ylim(0,50)

After 3 method, we conclude that using RandomClassifier gives the higher result. 

# Conclusion

After conducting the EDA, some notes need to be pointed out:

- The salary of most of vacancies is in the range of 110000-120000 dollar per year.
- The most popular professions are Data Engineer, Data Scientist, Data Analyst.
- The highest paid positions are Data Science Manager, Head of Data, Applied Scientist.
- Due to the nature of the work, employers require candidates with advanced work experience and full-time employment.
- Mid-sized companies hire more and pay higher wages than large and small companies.
- The US is the country with the most hiring companies and the average salary is also the highest. Employees working in companies based in India receive lower average salaries than in the other 9 countries compared.
- Data science positions are hiring more and more, and salaries are increasing every year.

After conducting statistical hypotheses, there are differences in wages between levels of the considered variables: type of employment, experience level, company size and year. This means that all levels of each variable make an effect on salary.

After using three methods, model used RandomForrestClassification algorithm gives the the highest score of accuracy.